In [1]:
# Enter your access token
access_token = ''

In [3]:
import requests
import pandas as pd
from datetime import datetime
from pandas.io.json import json_normalize

def fetch_and_process_enrollments(course_id, label):
    api_url = f"https://canvas.instructure.com/api/v1/courses/{course_id}/enrollments"
    headers = {"Authorization": f"Bearer {access_token}"}

    all_enrollments = []

    while api_url:
        response = requests.get(api_url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            all_enrollments.extend(data)

            next_page_url = response.links.get('next', None).get('url') if 'next' in response.links else None
            api_url = next_page_url
        else:
            print(f"Request failed with status code {response.status_code}: {response.text}")
            break

    # Flatten nested JSON structures
    flattened_data = pd.json_normalize(all_enrollments, sep='_')
    ## Print column names
    ##print(flattened_data.columns)

    # Define the columns you want to keep in the final DataFrame
    columns_to_keep = [
        'user_id', 'course_id', 'role','user_created_at', 'last_activity_at', 'total_activity_time',
        'user_name',  'user_login_id','grades_current_score', 'grades_final_score']

    # Select and reorder the columns
    df = flattened_data[columns_to_keep].copy()
    # Extract and format the date from the "user_created_at" column.
    df['user_created_at'] = df['user_created_at'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d") if x is not None else None)

    # Extract and format the date from the "last_activity_at" column.
    df['last_activity_at'] = df['last_activity_at'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d") if x is not None else None)
    # Save the DataFrame to a CSV file with the course ID and label in the name
    df.to_csv(f"course_{course_id}_{label}.csv", index=False)

    print(f"Enrollment data for course {course_id} ({label}) has been saved to enrollments_course_{course_id}_{label}.csv")

# List of course IDs and corresponding labels
course_info = [
    ('134780000000002817', 'Technical-Level'),
    ('134780000000002929', 'Managerial-Level'),
    ('134780000000002683', 'Course-3'),
    ('134780000000002735', 'Course-4')
]

# Fetch and process enrollments for each course
for course_id, label in course_info:
    fetch_and_process_enrollments(course_id, label)


Enrollment data for course 134780000000002817 (Technical-Level) has been saved to enrollments_course_134780000000002817_Technical-Level.csv
Enrollment data for course 134780000000002929 (Managerial-Level) has been saved to enrollments_course_134780000000002929_Managerial-Level.csv
Enrollment data for course 134780000000002683 (Enter Course Name) has been saved to enrollments_course_134780000000002683_Enter Course Name.csv
Enrollment data for course 134780000000002735 (Enter Course Name) has been saved to enrollments_course_134780000000002735_Enter Course Name.csv
